In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as pl
import sklearn.cross_validation as cv
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor,export_graphviz
from DecisionTree import DecisionTree
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import GradientBoostingRegressor
from GradientBoosting import GradientBoosting as myGB
# Plotting config
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
FOLDER = "data/"
FILES = [
        "iris.txt", "bezdekIris.txt", "wine.txt", "bupa.txt", "housing.txt", "auto-mpg.txt", "spam"
        ]
FILE = "spam"

In [3]:
# Подготавливаем признаки и целевую функцию
if FILE in FILES[:6]:
    
    df = pd.read_csv(FOLDER+FILE, sep=",", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
    
    if FILE in FILES[:2]:
        # ИРИСЫ
        df[4] = pd.factorize(df[4])[0]
        X = df[[0,1,2,3]].as_matrix()
        y = df[4].as_matrix()

    if FILE == FILES[2]:
        x_indexes = [x for x in range(1,14)]
        X = df[x_indexes].as_matrix()
        y = df[0].as_matrix()

    if FILE == FILES[3]:
        X = df[[0,1,2,3,4,5]].as_matrix()
        y = df[6]
        
    if FILE == FILES[4]:
        df = pd.read_csv(FOLDER+FILE, sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
        X = df[df.columns[1:]].as_matrix()
        y = df[df.columns[0]].as_matrix()
        
    if FILE == FILES[5]:
        df = pd.read_csv(FOLDER+FILE, sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
        X = df[df.columns[1:-1]].as_matrix()
        y = df[df.columns[0]].as_matrix()
        
    x_train, x_test, y_train, y_test = cv.train_test_split(X, y, test_size=0.25)

else:
    
    df_train = pd.read_csv(FOLDER+FILE+".train.txt", sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
    df_test = pd.read_csv(FOLDER+FILE+".test.txt", sep=" ", header=None)#, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)
    x_train = df_train[df_train.columns[1:]].as_matrix()
    y_train = df_train[df_train.columns[0]].as_matrix()
    x_test = df_test[df_test.columns[1:]].as_matrix()
    y_test = df_test[df_test.columns[0]].as_matrix()

# Дерево решений

### Классификация

In [ ]:
%%time
my_tree = DecisionTree(max_features=20, min_samples_leaf=2, max_steps=None, rsm=False)
my_tree.fit(x_train, y_train)
y_predicted = my_tree.predict(x_test)
print accuracy_score(y_test, y_predicted)

In [ ]:
%%time
sklearn_tree = DecisionTreeClassifier()
sklearn_tree.fit(x_train, y_train)
y_predicted = sklearn_tree.predict(x_test)
print accuracy_score(y_test, y_predicted)

### Регрессия

In [ ]:
my_tree = DecisionTree(is_classification=False, rsm=False, max_depth=3)
my_tree.fit(x_train, y_train)
y_predicted = my_tree.predict(x_test)
print mse(y_test, y_predicted)

In [ ]:
sklearn_tree = DecisionTreeRegressor(splitter='best', max_depth=3)
sklearn_tree.fit(x_train, y_train)
y_predicted = sklearn_tree.predict(x_test)
print mse(y_test, y_predicted)

In [ ]:
from Impurity.RegressionImpurity import RegressionImpurity

In [ ]:
x = x_train[0:,48]

In [ ]:
ri = RegressionImpurity()

In [ ]:
x_sort = x.argsort()
x = x[x_sort]
y = y_train[x_sort]
v = 0.01
ri.calculate_node(y[x<=v])+ri.calculate_node(y[x>v])

In [ ]:
np.unique(x)

In [ ]:
sum(x<=0.180471217255)

In [ ]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [ ]:
visualize_tree(sklearn_tree, range(0,102))

# Градиентный бустинг

### Регрессия

In [ ]:
#TODO защита от одинаковых данных

In [4]:
%%time
my_gb = myGB(n_estimators=200, max_depth=4, rsm=True, max_features=len(x_test[0]), shrinkage=0.1)
my_gb.fit(x_train, y_train)
y_predict = my_gb.predict(x_test)
print mse(y_test, y_predict)

0.0664961867195
CPU times: user 1min 19s, sys: 18.1 ms, total: 1min 19s
Wall time: 1min 19s


In [5]:
%%time
gb = GradientBoostingRegressor(n_estimators=200, max_depth=4, learning_rate=0.1)
gb.fit(x_train, y_train)
y_predicted = gb.predict(x_test)
print mse(y_test, y_predicted)

0.067441320764
CPU times: user 16.8 s, sys: 20 ms, total: 16.8 s
Wall time: 16.9 s
